In [ ]:
import pandas as pd
import numpy as np
import os
import glob
import zipfile
import pyarrow.parquet as pq
from functools import reduce
import matplotlib.pyplot as plt
import seaborn as sns
import warnings
from config import FAILED_BANK_LIST_FILE, CROSSWALK_CSV, UBPR_PANEL_FILE, REPORTS_DIR, PROCESSED_DATA_DIR




In [ ]:
warnings.filterwarnings('ignore')


# Read in failed bank list


In [ ]:
print("Reading Failed Bank List...")
failures = pd.read_csv(FAILED_BANK_LIST_FILE, encoding="cp1252")
failures = failures.rename(columns=lambda x: x.strip())
if "Cert\xa0" in failures.columns:
    failures = failures.rename(columns={"Cert\xa0": "CERT"})
elif "Cert" in failures.columns:
    failures = failures.rename(columns={"Cert": "CERT"})
failures["CERT"] = failures["CERT"].astype(str).str.strip()
print(failures.head())

Reading Failed Bank List...
                               Bank Name          City State   CERT  \
0           The Santa Anna National Bank    Santa Anna    TX   5520   
1                   Pulaski Savings Bank       Chicago    IL  28611   
2         First National Bank of Lindsay       Lindsay    OK   4134   
3  Republic First Bank dba Republic Bank  Philadelphia    PA  27332   
4                          Citizens Bank      Sac City    IA   8758   

               Acquiring Institution Closing Date   Fund  
0          Coleman County State Bank    27-Jun-25  10549  
1                    Millennium Bank    17-Jan-25  10548  
2             First Bank & Trust Co.    18-Oct-24  10547  
3  Fulton Bank, National Association    26-Apr-24  10546  
4          Iowa Trust & Savings Bank     3-Nov-23  10545  


# Read in FFIEC keys 

In [ ]:
print("\nReading FFIEC Keys (Crosswalk)...")
crosswalk = pd.read_csv(CROSSWALK_CSV)
crosswalk['ID_RSSD'] = crosswalk['ID_RSSD'].astype(str).str.strip()
crosswalk['CERT'] = crosswalk['CERT'].astype(str).str.strip()
print(crosswalk.head())


Reading FFIEC Keys (Crosswalk)...
  ID_RSSD   CERT
0      37  10057
1     242   3850
2     279  28868
3     354  14083
4     439  16498


# Read in UBPR data

In [ ]:
print("\nReading UBPR Data...")
table = pq.read_table(UBPR_PANEL_FILE)
ubpr = table.to_pandas(types_mapper=lambda pa_type: pd.StringDtype())
print(ubpr.head())


Reading UBPR Data...
         Reporting Period ID RSSD UBPRE006 UBPRE019 UBPRE020 UBPRE021  \
0  12/31/2002 11:59:59 PM      37     0.26     0.62     4.92     3.52   
1  12/31/2002 11:59:59 PM     242     0.07     0.11    23.16     6.74   
2  12/31/2002 11:59:59 PM     279     0.03     0.01    707.0    176.0   
3  12/31/2002 11:59:59 PM     354     0.21     0.38     5.38     8.44   
4  12/31/2002 11:59:59 PM     439     0.34     0.41     6.26     2.36   

  UBPRE022 UBPRE395 UBPR7316 UBPRE027  ... UBPRE003 UBPRE007 UBPRE009  \
0     2.26     8.42    11.56    -2.89  ...     4.45     1.83     1.84   
1      0.7     <NA>     2.05    17.04  ...     4.19     1.65     1.65   
2     1.23     <NA>    29.53     8.48  ...     2.76     1.24     1.19   
3      3.0     0.52     -1.7    12.17  ...     4.55     0.81     1.01   
4     0.95     0.75     2.83      2.4  ...     4.61     1.59     1.59   

  UBPRE010 UBPRE014 UBPRE015 UBPRE028 UBPRE029 UBPRKW07 UBPRPG69  
0     1.21    94.55     72.1   11

# Merge all datasets

In [ ]:
# Merge Datasets
print("\nMerging Datasets...")
# Convert Reporting Period to datetime for merging
ubpr['Reporting Period'] = pd.to_datetime(ubpr['Reporting Period'], errors='coerce')
failures['Closing Date'] = pd.to_datetime(failures['Closing Date'], errors='coerce')

# Merge UBPR with crosswalk to get CERT
model_data = pd.merge(ubpr, crosswalk, left_on='ID RSSD', right_on='ID_RSSD', how='left')

# Create a binary target: 1 if bank failed, 0 otherwise
model_data = pd.merge(model_data, failures[['CERT', 'Closing Date']], on='CERT', how='left')
model_data['Failed'] = data['Closing Date'].notna().astype(int)

print(model_data.head())



Merging Datasets...
     Reporting Period ID RSSD UBPRE006 UBPRE019 UBPRE020 UBPRE021 UBPRE022  \
0 2002-12-31 23:59:59      37     0.26     0.62     4.92     3.52     2.26   
1 2002-12-31 23:59:59     242     0.07     0.11    23.16     6.74      0.7   
2 2002-12-31 23:59:59     279     0.03     0.01    707.0    176.0     1.23   
3 2002-12-31 23:59:59     354     0.21     0.38     5.38     8.44      3.0   
4 2002-12-31 23:59:59     439     0.34     0.41     6.26     2.36     0.95   

  UBPRE395 UBPR7316 UBPRE027  ... UBPRE014 UBPRE015 UBPRE028 UBPRE029  \
0     8.42    11.56    -2.89  ...    94.55     72.1   118.16    60.81   
1     <NA>     2.05    17.04  ...    95.08    73.86   -18.98      9.3   
2     <NA>    29.53     8.48  ...    96.68    86.77    15.69    -6.15   
3     0.52     -1.7    12.17  ...    96.11     69.9   326.92    -30.5   
4     0.75     2.83      2.4  ...    91.16    67.97   -22.31      0.8   

  UBPRKW07 UBPRPG69 ID_RSSD   CERT Closing Date Failed  
0     <NA>    

In [ ]:
model_data.shape

(595829, 38)

# Organise data to improve readability

In [ ]:


model_data = model_data.sort_values(by=["ID RSSD", "Reporting Period"])

desired_order = [
    'Reporting Period', 'Closing Date', 'ID RSSD', 'ID_RSSD', 'CERT',
    "UBPRD486",
    "UBPRD488",
    "UBPR7402",
    "UBPRNC98",
    "UBPR7408",
    "UBPRE022",
    "UBPRE021",
    "UBPRE395",
    "UBPRE544",
    "UBPR7414",
    "UBPRE549",
    "UBPRE019",
    "UBPRE020",
    "UBPRE001",
    "UBPRE002",
    "UBPRE003",
    "UBPRE004",
    "UBPRE005",
    "UBPRPG69",
    "UBPRE006",
    "UBPRKW07",
    "UBPRE007",
    "UBPRE009",
    "UBPRE010",
    "UBPRE013",
    "UBPRK447",
    "UBPRE014",
    "UBPRE015",
    "UBPRE029",
    "UBPR7316",
    "UBPRE027",
    "UBPRE028"
]


model_data = model_data[desired_order]

print(model_data.head())


         Reporting Period Closing Date  ID RSSD  ID_RSSD   CERT UBPRD486  \
6706  2002-12-31 23:59:59          NaT  1000052  1000052  12241      8.2   
34968 2003-03-31 23:59:59          NaT  1000052  1000052  12241     8.37   
34969 2003-06-30 23:59:59          NaT  1000052  1000052  12241     7.94   
34970 2003-09-30 23:59:59          NaT  1000052  1000052  12241     8.19   
34967 2003-12-31 23:59:59          NaT  1000052  1000052  12241     8.12   

      UBPRD488 UBPR7402 UBPRNC98 UBPR7408  ... UBPRE009 UBPRE010 UBPRE013  \
6706     10.46    28.16     6.91    11.96  ...     1.84     1.22     1.22   
34968    11.07      0.0     8.32    11.44  ...     1.68     1.11     1.11   
34969    10.39    99.01     7.58     3.61  ...     1.71     1.13     1.13   
34970    11.04    65.23     5.72     7.31  ...     1.72     1.14     1.14   
34967    10.48    87.54     5.51     1.69  ...     1.64     1.08     1.08   

      UBPRK447 UBPRE014 UBPRE015 UBPRE029 UBPR7316 UBPRE027 UBPRE028  
6706     

In [ ]:
# Initialize target column to 0
model_data["FAIL_FLAG"] = 0

# For banks with a closing date, find the last reporting period
failed_banks = model_data.dropna(subset=["Closing Date"])["CERT"].unique()

for cert in failed_banks:
    bank_data = model_data[model_data["CERT"] == cert]
    
    # Last reporting period before/at closing date
    max_report = bank_data.loc[
        bank_data["Reporting Period"] <= bank_data["Closing Date"].iloc[0],
        "Reporting Period"
    ].max()
    
    # Mark that row as failure (1)
    model_data.loc[
        (model_data["CERT"] == cert) & 
        (model_data["Reporting Period"] == max_report),
        "FAIL_FLAG"
    ] = 1


print(model_data.head())

         Reporting Period Closing Date  ID RSSD  ID_RSSD   CERT UBPRD486  \
6706  2002-12-31 23:59:59          NaT  1000052  1000052  12241      8.2   
34968 2003-03-31 23:59:59          NaT  1000052  1000052  12241     8.37   
34969 2003-06-30 23:59:59          NaT  1000052  1000052  12241     7.94   
34970 2003-09-30 23:59:59          NaT  1000052  1000052  12241     8.19   
34967 2003-12-31 23:59:59          NaT  1000052  1000052  12241     8.12   

      UBPRD488 UBPR7402 UBPRNC98 UBPR7408  ... UBPRE010 UBPRE013 UBPRK447  \
6706     10.46    28.16     6.91    11.96  ...     1.22     1.22    13.79   
34968    11.07      0.0     8.32    11.44  ...     1.11     1.11    14.59   
34969    10.39    99.01     7.58     3.61  ...     1.13     1.13    18.34   
34970    11.04    65.23     5.72     7.31  ...     1.14     1.14     14.9   
34967    10.48    87.54     5.51     1.69  ...     1.08     1.08    15.63   

      UBPRE014 UBPRE015 UBPRE029 UBPR7316 UBPRE027 UBPRE028 FAIL_FLAG  
6706    

In [ ]:
rename_map = {
    "UBPRD486": "Tier1_Leverage_Ratio",
    "UBPRD488": "Total_Risk_Based_Capital_Ratio",
    "UBPR7402": "Cash_Dividends_to_Net_Income",
    "UBPRNC98": "Noncurrent_Loans_OREO_to_Tier1_ACL",
    "UBPR7408": "Tier1_Capital_Growth_Rate",
    "UBPRE022": "ACL_to_Loans_Leases_HFI",
    "UBPRE021": "ACL_to_Net_Loan_Losses",
    "UBPRE395": "ACL_to_Nonaccrual_Loans_Leases",
    "UBPRE544": "Loans_30_89_Days_Past_Due_Ratio",
    "UBPR7414": "Noncurrent_Loans_to_Gross_Loans",
    "UBPRE549": "Noncurrent_Loans_OREO_to_Loans_OREO",
    "UBPRE019": "Net_Loan_Losses_to_Avg_Loans",
    "UBPRE020": "Earnings_Coverage_of_Net_Losses",
    "UBPRE001": "Interest_Income_to_Avg_Assets",
    "UBPRE002": "Interest_Expense_to_Avg_Assets",
    "UBPRE003": "Net_Interest_Income_to_Avg_Assets",
    "UBPRE004": "Noninterest_Income_to_Avg_Assets",
    "UBPRE005": "Noninterest_Expense_to_Avg_Assets",
    "UBPRPG69": "Pre_Provision_Net_Revenue_to_Avg_Assets",
    "UBPRE006": "Provision_Loan_Losses_to_Avg_Assets",
    "UBPRKW07": "Provision_Other_Assets_Losses_to_Avg_Assets",
    "UBPRE007": "Pretax_Operating_Income_to_Avg_Assets",
    "UBPRE009": "Pretax_Net_Operating_Income_to_Avg_Assets",
    "UBPRE010": "Net_Operating_Income_to_Avg_Assets",
    "UBPRE013": "Net_Income_to_Avg_Assets",
    "UBPRK447": "Net_Noncore_Funding_Dependence",
    "UBPRE014": "Avg_Earning_Assets_to_Avg_Assets",
    "UBPRE015": "Avg_Interest_Bearing_Funds_to_Avg_Assets",
    "UBPRE029": "Short_Term_Noncore_Funding_Growth",
    "UBPR7316": "Total_Assets_Annual_Change",
    "UBPRE027": "Net_Loans_Leases_Growth_Rate",
    "UBPRE028": "Short_Term_Investments_Growth_Rate"
}


In [ ]:
data = model_data.rename(columns=rename_map)

In [ ]:
data.to_parquet(os.path.join(PROCESSED_DATA_DIR, 'analysis_data.parquet'), index=False)
print(f"Saved dataset to {os.path.join(PROCESSED_DATA_DIR, 'analysis_data.parquet')}")


Saved dataset to /Users/verena/Documents/Python_Projects/CreditModels/PD_FFIEC/data/processed/analysis_data.parquet
